In [1]:
DEVELOPMENT_MODE = False
try:
    import google.colab

    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/jbloomAus/SAELens

except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Jupyter notebook - intended for development only!


C:\Users\huohu\AppData\Local\Temp\ipykernel_18968\740336833.py:16: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("load_ext autoreload")
C:\Users\huohu\AppData\Local\Temp\ipykernel_18968\740336833.py:17: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")


In [2]:
import torch
if torch.cuda.is_available():
    device = "cuda"
# elif torch.backends.mps.is_available():
#     device = "mps"
else:
    device = "cpu"
torch.set_grad_enabled(False)

In [3]:
import sae_lens

d:\code\sae\SAE_Geometry\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import tqdm
from sae_lens import SAE
import numpy as np

hook_name_to_sae = []
length = 8013
d_sae = 24576
layers = 12
acts = torch.zeros((layers, length, d_sae)).to(device)
for layer in tqdm.tqdm(range(layers)):
    sae, cfg_dict, _ = SAE.from_pretrained(
        "gpt2-small-res-jb",
        f"blocks.{layer}.hook_resid_pre",
        device=device,
    )
    hook_name_to_sae.append(sae)
    with open(f"acts/activation0001_layer{layer}.npy", "rb") as f:
        for idx in range(length):
            acts[layer, idx] = torch.from_numpy(np.load(f)).to(device)

  0%|          | 0/12 [00:00<?, ?it/s]d:\code\sae\SAE_Geometry\.venv\Lib\site-packages\sae_lens\sae.py:145: UserWarning: 
This SAE has non-empty model_from_pretrained_kwargs. 
For optimal performance, load the model like so:
model = HookedSAETransformer.from_pretrained_no_processing(..., **cfg.model_from_pretrained_kwargs)
  warnings.warn(
100%|██████████| 12/12 [00:51<00:00,  4.33s/it]


In [5]:
from jaxtyping import Float
from torch import Tensor
import torch
import torch.nn.functional as F
def get_cosine_similarity(
    dict_elements_1: Float[Tensor, "n_dense n_dict_1"],
    dict_elements_2: Float[Tensor, "n_dense n_dict_2"],
    p_norm: int = 2,
    dim: int = 1,
) -> Float[Tensor, "n_dict_1 n_dict_2"]:
    """Get the cosine similarity between the alive dictionary elements of two runs.

    Args:
        dict_elements_1: The alive dictionary elements of the first run.
        dict_elements_2: The alive dictionary elements of the second run.

    Returns:
        The cosine similarity between the alive dictionary elements of the two runs.
    """
    # Compute cosine similarity in pytorch
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dict_elements_1 = dict_elements_1.to(device)
    dict_elements_2 = dict_elements_2.to(device)

    # Normalize the tensors
    dict_elements_1 = F.normalize(dict_elements_1, p=p_norm, dim=dim)
    dict_elements_2 = F.normalize(dict_elements_2, p=p_norm, dim=dim)

    # Compute cosine similarity using matrix multiplication
    cosine_sim: Float[Tensor, "n_dict_1 n_dict_2"] = torch.mm(dict_elements_1, dict_elements_2.T)
    return cosine_sim

In [6]:
from sae_lens import HookedSAETransformer

model: HookedSAETransformer = HookedSAETransformer.from_pretrained("gpt2-small").to(
    device
)

from datasets import load_dataset

# Load the Skylion007/openwebtext dataset
dataset = load_dataset('Skylion007/openwebtext')
length = 8013
# Print the first example from the dataset
doc_length = torch.zeros([length]).to(device)

for idx in tqdm.tqdm(range(length)):
#for idx in tqdm.tqdm(range(10)):
    example = dataset['train'][idx]
    tokens = model.to_tokens([example['text']], prepend_bos=True)
    doc_length[idx] = tokens.shape[1]

weights = F.normalize(doc_length, p=1, dim=0)
expanded_weights = weights.unsqueeze(0).unsqueeze(2).expand(12, 8013, 24576)
acts_weighted = (acts * expanded_weights).sum(dim=1)
acts_weighted.shape

Loaded pretrained model gpt2-small into HookedTransformer
Moving model to device:  cuda


d:\code\sae\SAE_Geometry\.venv\Lib\site-packages\datasets\load.py:1454: FutureWarning: The repository for Skylion007/openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Skylion007/openwebtext
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
100%|██████████| 8013/8013 [00:13<00:00, 591.12it/s]


torch.Size([12, 24576])

In [9]:
torch.save(acts_weighted, "data/acts_weighted.pt")

In [10]:
torch.save(weights, "data/weights.pt")

In [12]:
cos_sim = torch.zeros((layers, d_sae, d_sae))
for layer in tqdm.tqdm(range(12)):
    cos_sim[layer] = get_cosine_similarity(hook_name_to_sae[layer].W_dec, hook_name_to_sae[layer].W_dec).cpu()
    for length in range(d_sae):
        cos_sim[layer][length][length] = 0
cos_sim.shape
torch.save(cos_sim, "data/cos_sim.pt")

100%|██████████| 12/12 [00:07<00:00,  1.61it/s]
